<a href="https://colab.research.google.com/github/thunwaaa/sign_language/blob/main/model_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install library

In [ ]:
!pip install -q mediapipe opencv-python matplotlib tqdm open3d pandas

In [ ]:
!wget -q https://storage.googleapis.com/mediapipe-models/hand_landmarker/hand_landmarker/float16/1/hand_landmarker.task

## Connect to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### **สร้างโฟลเดอร์สำหรับเก็บผลลัพธ์**

In [ ]:
!mkdir -p '/content/drive/MyDrive/sign_language_3d_project/landmarks'
!mkdir -p '/content/drive/MyDrive/sign_language_3d_project/visualizations'
!mkdir -p '/content/drive/MyDrive/sign_language_3d_project/models'

In [ ]:
!ls "/content/drive/MyDrive" #check drive connect is complete or not?

In [ ]:
import os
import cv2
import mediapipe as mp
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
from tqdm.notebook import tqdm

In [ ]:
main_folder = '/content/drive/MyDrive/sign'
output_folder = '/content/drive/MyDrive/sign_language_3d_project/landmarks'

In [ ]:
# สำรวจโครงสร้างโฟลเดอร์
sign_words = os.listdir(main_folder)
print(f"พบคำภาษามือทั้งหมด: {len(sign_words)} คำ")

In [ ]:
# ตรวจสอบจำนวนวิดีโอในแต่ละโฟลเดอร์
for word in sign_words[:50]:  # แสดงตัวอย่าง 5 คำแรก
    word_folder = os.path.join(main_folder, word)
    videos = [f for f in os.listdir(word_folder) if f.endswith(('.mp4', '.avi', '.mov', '.MP4', '.MOV'))]
    print(f"คำ '{word}' มีวิดีโอ {len(videos)} คลิป")

In [ ]:
#settings mediapipe
mp_hands = mp.solutions.hands
mp_pose = mp.solutions.pose
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

In [ ]:
def extract_landmarks_from_video(video_path, max_frames=30, extract_face=True):
    """
    สกัดจุดสำคัญจากวิดีโอโดยใช้ MediaPipe

    Args:
        video_path (str): พาธไปยังไฟล์วิดีโอ
        max_frames (int): จำนวนเฟรมสูงสุดที่จะสกัด
        extract_face (bool): เปิดใช้การสกัดใบหน้าแบบละเอียดหรือไม่

    Returns:
        list: รายการข้อมูลจุดสำคัญในแต่ละเฟรม
    """
    cap = cv2.VideoCapture(video_path)
    frames_landmarks = []

    # ตรวจสอบว่าเปิดวิดีโอได้หรือไม่
    if not cap.isOpened():
        print(f"ไม่สามารถเปิดวิดีโอ: {video_path}")
        return frames_landmarks

    # ดึงข้อมูลวิดีโอ
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    duration = frame_count / fps if fps > 0 else 0
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    print(f"วิดีโอ: {os.path.basename(video_path)}")
    print(f"  ความละเอียด: {width}x{height}")
    print(f"  จำนวนเฟรม: {frame_count}")
    print(f"  FPS: {fps}")
    print(f"  ความยาว: {duration:.2f} วินาที")

    # คำนวณความถี่ในการสุ่มเฟรม (เพื่อให้ได้ max_frames เฟรม)
    sample_interval = max(1, frame_count // max_frames)

    # MediaPipe models
    with mp_hands.Hands(
        static_image_mode=False,
        max_num_hands=2,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as hands, \
        mp_pose.Pose(
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5) as pose:

        # เพิ่ม Face Mesh ถ้าต้องการ
        face_mesh = None
        if extract_face:
            face_mesh = mp_face_mesh.FaceMesh(
                static_image_mode=False,
                max_num_faces=1,
                min_detection_confidence=0.5,
                min_tracking_confidence=0.5)

        frame_idx = 0
        pbar = tqdm(total=min(frame_count, max_frames), desc="กำลังประมวลผลเฟรม")

        while cap.isOpened() and len(frames_landmarks) < max_frames:
            success, frame = cap.read()
            if not success:
                break

            # สุ่มเฟรมตาม sample_interval
            if frame_idx % sample_interval != 0:
                frame_idx += 1
                continue

            # แปลงสี BGR เป็น RGB (MediaPipe ต้องการ RGB)
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # ตรวจจับมือและท่าทาง
            hand_results = hands.process(frame_rgb)
            pose_results = pose.process(frame_rgb)

            # ตรวจจับใบหน้า (ถ้าเปิดใช้)
            face_results = None
            if face_mesh:
                face_results = face_mesh.process(frame_rgb)

            # สกัดข้อมูลจุดสำคัญ
            frame_data = {
                'frame_idx': frame_idx,
                'timestamp': frame_idx / fps if fps > 0 else 0,
                'hands': [],
                'pose': None,
                'face': None
            }

            # เก็บข้อมูลมือ
            if hand_results.multi_hand_landmarks and hand_results.multi_handedness:
                for i, (hand_landmarks, handedness) in enumerate(zip(
                    hand_results.multi_hand_landmarks, hand_results.multi_handedness)):

                    # ตรวจสอบว่าเป็นมือซ้ายหรือขวา
                    hand_type = handedness.classification[0].label
                    is_right = (hand_type == "Right")

                    # เก็บข้อมูลจุดสำคัญ
                    hand_data = []
                    for landmark in hand_landmarks.landmark:
                        hand_data.append([landmark.x, landmark.y, landmark.z])

                    # เพิ่มข้อมูลมือ
                    frame_data['hands'].append({
                        'landmarks': hand_data,
                        'is_right': is_right
                    })

            # เก็บข้อมูลท่าทาง
            if pose_results.pose_landmarks:
                pose_data = []
                for landmark in pose_results.pose_landmarks.landmark:
                    pose_data.append([landmark.x, landmark.y, landmark.z])
                frame_data['pose'] = pose_data

            # เก็บข้อมูลใบหน้า (ถ้ามี)
            if face_mesh and face_results.multi_face_landmarks:
                face_data = []
                for face_landmarks in face_results.multi_face_landmarks:
                    for landmark in face_landmarks.landmark:
                        face_data.append([landmark.x, landmark.y, landmark.z])
                    # เก็บเฉพาะใบหน้าแรก
                    break
                frame_data['face'] = face_data

            # เพิ่มข้อมูลเฟรมนี้
            frames_landmarks.append(frame_data)
            frame_idx += 1
            pbar.update(1)

        pbar.close()

    # ปิดวิดีโอ
    cap.release()

    if face_mesh:
        face_mesh.close()

    print(f"สกัดข้อมูลสำเร็จ: {len(frames_landmarks)} เฟรม")
    return frames_landmarks

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

Support for third party widgets will remain active for the duration of the session. To disable support:

In [ ]:
# โหลดข้อมูลสรุป
with open(os.path.join(output_folder, "summary.json"), 'r', encoding='utf-8') as f:
    summary_data = json.load(f)

# แสดงจำนวนวิดีโอในแต่ละคำ
for word, count in summary_data.items():
    print(f"คำ '{word}': {count} วิดีโอ")

# โหลดข้อมูลของคำแรกเพื่อวิเคราะห์เพิ่มเติม
first_word = list(summary_data.keys())[0]
with open(os.path.join(output_folder, f"{first_word}.json"), 'r', encoding='utf-8') as f:
    first_word_data = json.load(f)

# ตรวจสอบจำนวนเฟรมในแต่ละวิดีโอ
for i, video_data in enumerate(first_word_data):
    print(f"วิดีโอที่ {i+1}: {video_data['file_name']} - {len(video_data['landmarks'])} เฟรม")

In [ ]:
from google.colab import output
output.disable_custom_widget_manager()

### ปรับปรุงการแสดงผลเพื่อตรวจสอบการจับจุดสำคัญ

In [ ]:
def visualize_hand_tracking_improved(input_video_path, output_video_path=None, show_face=True):
    """
    สร้างวิดีโอที่แสดงการตรวจจับจุดสำคัญพร้อมหมายเลขจุดกำกับ

    Args:
        input_video_path: พาธไปยังวิดีโอต้นฉบับ
        output_video_path: พาธสำหรับบันทึกวิดีโอผลลัพธ์
        show_face: แสดงจุดสำคัญของใบหน้าหรือไม่

    Returns:
        str: พาธของวิดีโอผลลัพธ์
    """
    # กำหนดพาธสำหรับบันทึกวิดีโอผลลัพธ์
    if output_video_path is None:
        file_name, file_ext = os.path.splitext(input_video_path)
        output_video_path = f"{file_name}_visualized{file_ext}"

    # MediaPipe
    mp_drawing = mp.solutions.drawing_utils
    mp_drawing_styles = mp.solutions.drawing_styles
    mp_hands = mp.solutions.hands
    mp_pose = mp.solutions.pose
    mp_face_mesh = mp.solutions.face_mesh

    # เปิดวิดีโอต้นฉบับ
    cap = cv2.VideoCapture(input_video_path)
    if not cap.isOpened():
        print(f"ไม่สามารถเปิดวิดีโอ: {input_video_path}")
        return None

    # อ่านข้อมูลวิดีโอ
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # สร้าง VideoWriter สำหรับบันทึกวิดีโอผลลัพธ์
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    # ตัวเลือกการวาด
    hand_drawing_spec = mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2)
    hand_connection_spec = mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2)

    # MediaPipe models
    with mp_hands.Hands(
        static_image_mode=False,
        max_num_hands=2,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as hands, \
        mp_pose.Pose(
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5) as pose, \
        mp_face_mesh.FaceMesh(
            static_image_mode=False,
            max_num_faces=1,
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5) as face_mesh:

        # วนลูปผ่านทุกเฟรมในวิดีโอ
        pbar = tqdm(total=total_frames, desc="กำลังประมวลผลวิดีโอ")
        frame_count = 0

        while cap.isOpened():
            success, image = cap.read()
            if not success:
                break

            # แปลงภาพเป็น RGB สำหรับ MediaPipe
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # ประมวลผลภาพด้วย MediaPipe
            hand_results = hands.process(image_rgb)
            pose_results = pose.process(image_rgb)
            face_results = face_mesh.process(image_rgb) if show_face else None

            # วาดข้อมูลบนภาพ
            annotated_image = image.copy()

            # แสดงเฟรมเคาท์เตอร์
            cv2.putText(
                annotated_image,
                f"Frame: {frame_count}",
                (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (0, 255, 255),
                2
            )

            # วาดจุดสำคัญและเส้นเชื่อมต่อของมือ
            if hand_results.multi_hand_landmarks:
                for i, hand_landmarks in enumerate(hand_results.multi_hand_landmarks):
                    # วาดจุดและเส้นเชื่อมต่อ
                    mp_drawing.draw_landmarks(
                        annotated_image,
                        hand_landmarks,
                        mp_hands.HAND_CONNECTIONS,
                        hand_drawing_spec,
                        hand_connection_spec)

                    # วาดหมายเลขกำกับจุดสำคัญ
                    for id, lm in enumerate(hand_landmarks.landmark):
                        h, w, c = annotated_image.shape
                        cx, cy = int(lm.x * w), int(lm.y * h)
                        cv2.putText(
                            annotated_image,
                            str(id),
                            (cx, cy),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            0.5,
                            (255, 255, 0),
                            1
                        )

                    # ระบุว่าเป็นมือซ้ายหรือขวา
                    if i < len(hand_results.multi_handedness):
                        handedness = hand_results.multi_handedness[i]
                        hand_type = handedness.classification[0].label
                        h, w, c = annotated_image.shape
                        wrist = hand_landmarks.landmark[0]
                        cx, cy = int(wrist.x * w), int(wrist.y * h)
                        cv2.putText(
                            annotated_image,
                            hand_type,
                            (cx, cy - 20),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            0.7,
                            (255, 0, 0),
                            2
                        )

            # วาดจุดสำคัญและเส้นเชื่อมต่อของท่าทาง
            if pose_results.pose_landmarks:
                mp_drawing.draw_landmarks(
                    annotated_image,
                    pose_results.pose_landmarks,
                    mp_pose.POSE_CONNECTIONS,
                    mp_drawing_styles.get_default_pose_landmarks_style())

                # วาดหมายเลขกำกับเฉพาะจุดสำคัญของแขนและลำตัวส่วนบน
                for id, lm in enumerate(pose_results.pose_landmarks.landmark):
                    if id in range(11, 23):  # แขนและลำตัวส่วนบน
                        h, w, c = annotated_image.shape
                        cx, cy = int(lm.x * w), int(lm.y * h)
                        cv2.putText(
                            annotated_image,
                            str(id),
                            (cx, cy),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            0.5,
                            (0, 0, 255),
                            1
                        )

            # วาดใบหน้า (ถ้าเปิดใช้)
            if show_face and face_results.multi_face_landmarks:
                for face_landmarks in face_results.multi_face_landmarks:
                    mp_drawing.draw_landmarks(
                        image=annotated_image,
                        landmark_list=face_landmarks,
                        connections=mp_face_mesh.FACEMESH_TESSELATION,
                        landmark_drawing_spec=None,
                        connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style())

            # บันทึกเฟรม
            out.write(annotated_image)
            frame_count += 1
            pbar.update(1)

        pbar.close()

    # ปิดการเชื่อมต่อ
    cap.release()
    out.release()

    print(f"บันทึกวิดีโอผลลัพธ์ไว้ที่: {output_video_path}")
    return output_video_path

### สร้างฟังก์ชันประมวลผลวิดีโอทั้งหมด

In [ ]:
def process_all_videos_improved(main_folder, output_folder, max_frames=30, extract_face=True,
                               skip_processed=True, batch_size=None):
    """
    ประมวลผลวิดีโอทั้งหมดและบันทึกข้อมูลจุดสำคัญ

    Args:
        main_folder: โฟลเดอร์ที่เก็บวิดีโอภาษามือ
        output_folder: โฟลเดอร์สำหรับบันทึกผลลัพธ์
        max_frames: จำนวนเฟรมสูงสุดต่อวิดีโอ
        extract_face: เปิดใช้การสกัดใบหน้าแบบละเอียดหรือไม่
        skip_processed: ข้ามคำที่ประมวลผลแล้วหรือไม่
        batch_size: จำนวนคำที่จะประมวลผลในแต่ละรอบ (None = ทั้งหมด)

    Returns:
        dict: ข้อมูลสรุป
    """
    # สร้างโฟลเดอร์สำหรับเก็บผลลัพธ์ถ้ายังไม่มี
    os.makedirs(output_folder, exist_ok=True)
    vis_folder = '/content/drive/MyDrive/sign_language_3d_project/visualizations'
    os.makedirs(vis_folder, exist_ok=True)

    # โหลดข้อมูลสรุปที่มีอยู่เดิม (ถ้ามี)
    summary_file = os.path.join(output_folder, "summary.json")
    processed_words = {}
    if os.path.exists(summary_file) and skip_processed:
        try:
            with open(summary_file, 'r', encoding='utf-8') as f:
                processed_words = json.load(f)
            print(f"พบข้อมูลสรุปเดิม: มีคำที่ประมวลผลแล้ว {len(processed_words)} คำ")
        except Exception as e:
            print(f"เกิดข้อผิดพลาดในการโหลดข้อมูลสรุมเดิม: {str(e)}")
            processed_words = {}

    # รายการคำทั้งหมด (เฉพาะโฟลเดอร์)
    sign_words = [d for d in os.listdir(main_folder)
                 if os.path.isdir(os.path.join(main_folder, d))]

    total_words = len(sign_words)
    print(f"พบคำทั้งหมด: {total_words} คำ")

    # กำหนดขนาดแบทช์ถ้าไม่ได้ระบุ
    if batch_size is None:
        batch_size = total_words

    all_data = processed_words.copy()  # เก็บข้อมูลทั้งหมด (รวมที่ประมวลผลไปแล้ว)

    # คำนวณจำนวนแบทช์
    num_batches = (total_words + batch_size - 1) // batch_size

    # ประมวลผลทีละแบทช์
    for batch_idx in range(num_batches):
        start_idx = batch_idx * batch_size
        end_idx = min((batch_idx + 1) * batch_size, total_words)

        print(f"\n=== ประมวลผลแบทช์ {batch_idx + 1}/{num_batches}: คำที่ {start_idx + 1} ถึง {end_idx} ===")

        # คำในแบทช์นี้
        batch_words = sign_words[start_idx:end_idx]

        # วนลูปผ่านทุกคำในแบทช์
        for word in tqdm(batch_words, desc=f"Processing words in batch {batch_idx + 1}"):
            # ข้ามคำที่ประมวลผลแล้ว
            if word in processed_words and skip_processed:
                print(f"\nข้ามคำ '{word}' เนื่องจากประมวลผลไปแล้ว")
                continue

            print(f"\nกำลังประมวลผลคำ: '{word}'")

            word_folder = os.path.join(main_folder, word)
            videos = [f for f in os.listdir(word_folder)
                     if f.lower().endswith(('.mp4', '.avi', '.mov', '.MP4', '.MOV'))]

            word_data = []  # เก็บข้อมูลของคำนี้

            # วนลูปผ่านทุกวิดีโอของคำนี้
            for video_file in tqdm(videos, desc=f"Videos for {word}", leave=False):
                video_path = os.path.join(word_folder, video_file)
                print(f"\nกำลังประมวลผลวิดีโอ: {video_file}")

                try:
                    # สกัดจุดสำคัญจากวิดีโอ
                    landmarks = extract_landmarks_from_video(
                        video_path,
                        max_frames=max_frames,
                        extract_face=extract_face
                    )

                    # เพิ่มข้อมูลเข้าไปในรายการของคำนี้
                    video_data = {
                        "file_name": video_file,
                        "landmarks": landmarks
                    }
                    word_data.append(video_data)

                    # สร้างวิดีโอแสดงผลการจับจุดสำคัญ
                    word_vis_folder = os.path.join(vis_folder, word)
                    os.makedirs(word_vis_folder, exist_ok=True)

                    output_video_path = os.path.join(word_vis_folder, f"{video_file}_viz.mp4")
                    visualize_hand_tracking_improved(
                        video_path,
                        output_video_path,
                        show_face=extract_face
                    )

                except Exception as e:
                    print(f"เกิดข้อผิดพลาดกับวิดีโอ {video_file}: {str(e)}")

            # บันทึกข้อมูลของคำนี้
            all_data[word] = len(word_data)

            # บันทึกข้อมูลแยกตามคำ
            word_output_file = os.path.join(output_folder, f"{word}.json")
            with open(word_output_file, 'w', encoding='utf-8') as f:
                json.dump(word_data, f)

            # อัปเดตและบันทึกข้อมูลสรุปหลังจากประมวลผลแต่ละคำ (ป้องกันการหลุดการเชื่อมต่อ)
            with open(summary_file, 'w', encoding='utf-8') as f:
                json.dump(all_data, f)

            print(f"บันทึกข้อมูลสำหรับคำ '{word}' เรียบร้อย: {len(word_data)} วิดีโอ")
            print(f"ความคืบหน้า: {len(all_data)}/{total_words} คำ ({len(all_data)/total_words*100:.2f}%)")

    print(f"\nประมวลผลเสร็จสิ้น! บันทึกข้อมูลไว้ที่ {output_folder}")
    print(f"จำนวนคำที่ประมวลผล: {len(all_data)}")

    return all_data

### ทดสอบการประมวลผลกับวิดีโอตัวอย่าง

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
# ทดสอบกับวิดีโอตัวอย่าง 3 คำ คำละ 2 วิดีโอ
test_output_folder = '/content/drive/MyDrive/sign_language_3d_project/landmarks'
all_data = process_all_videos(
    main_folder=main_folder,
    output_folder=test_output_folder,
    num_words=3,              # เริ่มต้นด้วย 3 คำ
    max_videos_per_word=2,    # แต่ละคำใช้ 2 วิดีโอ
    max_frames=30,            # แต่ละวิดีโอเก็บ 30 เฟรม
    extract_face=True         # เก็บข้อมูลใบหน้าด้วย
)

# แสดงภาพตัวอย่างวิดีโอที่ประมวลผลแล้ว
from IPython.display import HTML
from base64 import b64encode

def display_video(video_path):
    """แสดงวิดีโอใน Colab"""
    with open(video_path, "rb") as f:
        video_file = f.read()
    video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
    return HTML(f"""<video width=640 controls><source src="{video_url}"></video>""")

# เลือกวิดีโอแรกของคำแรกมาแสดง
first_word = list(all_data.keys())[0]
first_video = all_data[first_word][0]['file_name']
vis_video_path = f"/content/drive/MyDrive/sign_language_3d_project/visualizations/{first_word}/{first_video}_viz.mp4"
display_video(vis_video_path)

Support for third party widgets will remain active for the duration of the session. To disable support:

In [ ]:
from google.colab import output
output.disable_custom_widget_manager()

# เตรียมข้อมูลสำหรับโมเดล 3D

In [ ]:
def prepare_data_for_3d_model(landmarks_data):
    """
    แปลงข้อมูลให้พร้อมใช้กับโมเดล 3D

    Args:
        landmarks_data: ข้อมูลจุดสำคัญที่สกัดได้

    Returns:
        dict: ข้อมูลที่จัดรูปแบบสำหรับโมเดล 3D
    """
    processed_data = []

    for frame_data in landmarks_data:
        frame_3d_data = {
            'frame_idx': frame_data.get('frame_idx', 0),
            'timestamp': frame_data.get('timestamp', 0),
            'hands': [],
            'pose': None,
            'face': None
        }

        # แปลงข้อมูลมือ
        for hand in frame_data['hands']:
            hand_landmarks = hand['landmarks']
            is_right = hand['is_right']

            # จัดกลุ่มจุดตามส่วนของมือ
            hand_3d = {
                'is_right': is_right,
                'wrist': hand_landmarks[0],
                'thumb': hand_landmarks[1:5],
                'index': hand_landmarks[5:9],
                'middle': hand_landmarks[9:13],
                'ring': hand_landmarks[13:17],
                'pinky': hand_landmarks[17:21],
                'palm': [hand_landmarks[0], hand_landmarks[5], hand_landmarks[9], hand_landmarks[13], hand_landmarks[17]]
            }

            frame_3d_data['hands'].append(hand_3d)

        # แปลงข้อมูลท่าทาง
        if frame_data['pose']:
            pose = frame_data['pose']
            pose_3d = {
                'head': pose[0:11],
                'torso': [pose[11], pose[12], pose[23], pose[24]],
                'left_arm': [pose[11], pose[13], pose[15], pose[17], pose[19], pose[21]],
                'right_arm': [pose[12], pose[14], pose[16], pose[18], pose[20], pose[22]],
                'left_leg': [pose[23], pose[25], pose[27], pose[29], pose[31]],
                'right_leg': [pose[24], pose[26], pose[28], pose[30], pose[32]]
            }

            frame_3d_data['pose'] = pose_3d

        # แปลงข้อมูลใบหน้า (ถ้ามี)
        if frame_data.get('face'):
            # เลือกจุดสำคัญหลักของใบหน้า (ลดจำนวนจุดลง)
            # เลือกเฉพาะจุดรอบตา จมูก ปาก และขอบหน้า
            face = frame_data['face']
            face_3d = {
                'contour': [face[i] for i in range(0, 468, 20)],  # เลือกบางจุดเท่านั้น
                'left_eye': [face[33], face[133], face[160], face[159], face[158], face[144], face[145], face[153]],
                'right_eye': [face[263], face[362], face[385], face[386], face[387], face[373], face[374], face[380]],
                'nose': [face[1], face[2], face[3], face[4], face[5], face[6], face[168], face[197], face[195]],
                'mouth': [face[0], face[267], face[269], face[270], face[409], face[291], face[375], face[321], face[405], face[314], face[17], face[84], face[181], face[91], face[146]]
            }

            frame_3d_data['face'] = face_3d

        processed_data.append(frame_3d_data)

    return processed_data

###  บันทึกข้อมูลที่แปลงแล้วสำหรับโมเดล 3D

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
def export_3d_ready_data_improved(input_folder, output_folder, skip_processed=True, batch_size=None):
    """
    แปลงข้อมูลและบันทึกในรูปแบบที่พร้อมใช้กับโมเดล 3D

    Args:
        input_folder: โฟลเดอร์ที่เก็บข้อมูลจุดสำคัญดิบ
        output_folder: โฟลเดอร์สำหรับบันทึกข้อมูลที่แปลงแล้ว
        skip_processed: ข้ามคำที่ประมวลผลแล้วหรือไม่
        batch_size: จำนวนคำที่จะประมวลผลในแต่ละรอบ (None = ทั้งหมด)
    """
    os.makedirs(output_folder, exist_ok=True)

    # โหลดข้อมูลสรุป
    summary_file = os.path.join(input_folder, "summary.json")
    with open(summary_file, 'r', encoding='utf-8') as f:
        summary_data = json.load(f)

    # โหลดข้อมูลสรุปที่แปลงแล้ว (ถ้ามี)
    processed_summary_file = os.path.join(output_folder, "summary_3d.json")
    processed_words = {}
    if os.path.exists(processed_summary_file) and skip_processed:
        try:
            with open(processed_summary_file, 'r', encoding='utf-8') as f:
                processed_words = json.load(f)
            print(f"พบข้อมูลสรุป 3D เดิม: มีคำที่แปลงแล้ว {len(processed_words)} คำ")
        except Exception as e:
            print(f"เกิดข้อผิดพลาดในการโหลดข้อมูลสรุม 3D เดิม: {str(e)}")
            processed_words = {}

    # สร้างข้อมูลสรุปใหม่
    new_summary = processed_words.copy()

    # รายการคำทั้งหมด
    all_words = list(summary_data.keys())
    total_words = len(all_words)
    print(f"พบคำทั้งหมด: {total_words} คำ")

    # กำหนดขนาดแบทช์ถ้าไม่ได้ระบุ
    if batch_size is None:
        batch_size = total_words

    # คำนวณจำนวนแบทช์
    num_batches = (total_words + batch_size - 1) // batch_size

    # ประมวลผลทีละแบทช์
    for batch_idx in range(num_batches):
        start_idx = batch_idx * batch_size
        end_idx = min((batch_idx + 1) * batch_size, total_words)

        print(f"\n=== แปลงข้อมูลแบทช์ {batch_idx + 1}/{num_batches}: คำที่ {start_idx + 1} ถึง {end_idx} ===")

        # คำในแบทช์นี้
        batch_words = all_words[start_idx:end_idx]

        # แปลงข้อมูลทุกคำในแบทช์
        for word in tqdm(batch_words, desc=f"Converting data for 3D model in batch {batch_idx + 1}"):
            # ข้ามคำที่แปลงแล้ว
            if word in processed_words and skip_processed:
                print(f"\nข้ามคำ '{word}' เนื่องจากแปลงแล้ว")
                continue

            print(f"\nกำลังแปลงข้อมูลคำ: '{word}'")

            try:
                # โหลดข้อมูลคำ
                word_file = os.path.join(input_folder, f"{word}.json")
                if not os.path.exists(word_file):
                    print(f"ไม่พบไฟล์ข้อมูลสำหรับคำ '{word}'")
                    continue

                with open(word_file, 'r', encoding='utf-8') as f:
                    word_data = json.load(f)

                word_3d_data = []

                # แปลงข้อมูลทุกวิดีโอของคำนี้
                for video_data in tqdm(word_data, desc=f"Videos for {word}", leave=False):
                    video_name = video_data['file_name']
                    landmarks = video_data['landmarks']

                    # แปลงข้อมูลให้พร้อมใช้กับโมเดล 3D
                    processed_landmarks = prepare_data_for_3d_model(landmarks)

                    # เพิ่มข้อมูลที่แปลงแล้ว
                    video_3d_data = {
                        "file_name": video_name,
                        "frames": processed_landmarks
                    }
                    word_3d_data.append(video_3d_data)

                # บันทึกข้อมูลที่แปลงแล้วของคำนี้
                output_file = os.path.join(output_folder, f"{word}_3d.json")
                with open(output_file, 'w', encoding='utf-8') as f:
                    json.dump(word_3d_data, f)

                new_summary[word] = len(word_3d_data)

                # อัปเดตและบันทึกข้อมูลสรุปหลังจากแปลงแต่ละคำ (ป้องกันการหลุดการเชื่อมต่อ)
                with open(processed_summary_file, 'w', encoding='utf-8') as f:
                    json.dump(new_summary, f)

                print(f"บันทึกข้อมูล 3D สำหรับคำ '{word}' เรียบร้อย: {len(word_3d_data)} วิดีโอ")
                print(f"ความคืบหน้า: {len(new_summary)}/{total_words} คำ ({len(new_summary)/total_words*100:.2f}%)")

            except Exception as e:
                print(f"เกิดข้อผิดพลาดในการแปลงข้อมูลคำ '{word}': {str(e)}")

    print(f"\nการแปลงข้อมูลเสร็จสิ้น! บันทึกข้อมูลไว้ที่ {output_folder}")
    print(f"จำนวนคำที่แปลง: {len(new_summary)}")

    return new_summary

Support for third party widgets will remain active for the duration of the session. To disable support:

In [ ]:
from google.colab import output
output.disable_custom_widget_manager()

In [ ]:
# ประมวลผลวิดีโอทั้งหมดในครั้งเดียว
main_folder = '/content/drive/MyDrive/sign'
output_folder = '/content/drive/MyDrive/sign_language_3d_project/landmarks'
all_data = process_all_videos_improved(
    main_folder=main_folder,
    output_folder=output_folder,
    max_frames=30,
    extract_face=True,
    skip_processed=True  # ข้ามคำที่เคยประมวลผลแล้ว
)

# แปลงข้อมูลทั้งหมดเป็นรูปแบบ 3D ในครั้งเดียว
landmarks_folder = '/content/drive/MyDrive/sign_language_3d_project/landmarks'
output_3d_folder = '/content/drive/MyDrive/sign_language_3d_project/3d_data'
export_3d_ready_data_improved(
    input_folder=landmarks_folder,
    output_folder=output_3d_folder,
    skip_processed=True  # ข้ามคำที่เคยแปลงแล้ว
)

# ตรวจสอบข้อมูลที่แปลงแล้ว

In [ ]:
# โหลดและตรวจสอบข้อมูลที่แปลงแล้ว
def verify_3d_data(folder_path):
    """ตรวจสอบความถูกต้องของข้อมูลที่แปลงแล้ว"""
    # โหลดข้อมูลสรุป
    with open(os.path.join(folder_path, "summary_3d.json"), 'r', encoding='utf-8') as f:
        summary = json.load(f)

    print(f"จำนวนคำทั้งหมด: {len(summary)}")

    # เลือกคำแรกเพื่อตรวจสอบ
    first_word = list(summary.keys())[0]
    print(f"\nกำลังตรวจสอบคำ: '{first_word}'")

    # โหลดข้อมูลคำ
    with open(os.path.join(folder_path, f"{first_word}_3d.json"), 'r', encoding='utf-8') as f:
        word_data = json.load(f)

    print(f"จำนวนวิดีโอของคำนี้: {len(word_data)}")

    # เลือกวิดีโอแรกเพื่อตรวจสอบ
    first_video = word_data[0]
    print(f"\nชื่อวิดีโอ: {first_video['file_name']}")
    print(f"จำนวนเฟรม: {len(first_video['frames'])}")

    # ตรวจสอบเฟรมแรก
    first_frame = first_video['frames'][0]
    print("\nโครงสร้างข้อมูลเฟรมแรก:")

    # ตรวจสอบข้อมูลมือ
    print(f"จำนวนมือ: {len(first_frame['hands'])}")
    if first_frame['hands']:
        hand = first_frame['hands'][0]
        print(f"ประเภทมือ: {'ขวา' if hand['is_right'] else 'ซ้าย'}")
        print(f"จำนวนจุดสำคัญนิ้วหัวแม่มือ: {len(hand['thumb'])}")

    # ตรวจสอบข้อมูลท่าทาง
    if first_frame['pose']:
        print("\nข้อมูลท่าทาง:")
        for part, points in first_frame['pose'].items():
            print(f"  {part}: {len(points)} จุด")

    # ตรวจสอบข้อมูลใบหน้า
    if first_frame['face']:
        print("\nข้อมูลใบหน้า:")
        for part, points in first_frame['face'].items():
            print(f"  {part}: {len(points)} จุด")

    return first_video  # ส่งคืนข้อมูลวิดีโอแรกเพื่อใช้ในการทดสอบต่อไป

# ทดสอบการตรวจสอบข้อมูล
first_video_3d = verify_3d_data(output_3d_folder)